# LangChain in Notebook

Example based on the following resources:

* https://python.langchain.com/v0.2/docs/tutorials/qa_chat_history/
* https://quickstarts.snowflake.com/guide/asking_questions_to_your_own_documents_with_snowflake_cortex/index.html?index=..%2F..index#0

The usual one: Snowflake session.

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

Let us define some variables:

In [ ]:
MODEL_EMBEDDINGS = "e5-base-v2"
VECTOR_LENGTH = 768
MODEL_LLM = "llama3-70b"

DEBUG_LANGCHAIN = False

In [ ]:
import langchain

langchain.debug = DEBUG_LANGCHAIN

Use `SQLCortex` LLM

In [ ]:
from langchain_snowpoc.llms import SQLCortex

llm = SQLCortex(session=session, model=MODEL_LLM)

Use `SnowflakeEmbeddings` and `SnowflakeVectorStore`

In [ ]:
from langchain_snowpoc.embedding import SnowflakeEmbeddings

embeddings = SnowflakeEmbeddings(
    session=session, model=MODEL_EMBEDDINGS
)

In [ ]:
from langchain_snowpoc.vectorstores import SnowflakeVectorStore

v = SnowflakeVectorStore(
    table="LANGCHAIN",
    session=session,
    embedding=embeddings,
    vector_length=VECTOR_LENGTH
)
retriever = v.as_retriever()

Setup history support for LangChain

In [ ]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

Prompt for answering questions

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

### Answer question ###
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

And a RAG chain:

In [ ]:
rag_chain = create_retrieval_chain(
    history_aware_retriever,
    question_answer_chain
)

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

import uuid
from langchain_snowpoc.chat_message_histories import SnowflakeChatMessageHistory

## setup chat history
table_name = "chat_history"
SnowflakeChatMessageHistory.create_tables(session, table_name)


### Statefully manage chat history ###
def get_session_history_wrap(
    session,
    table_name: str = 'chat_history'):
    def _get_session_history(session_id: str, ) -> BaseChatMessageHistory:
        return SnowflakeChatMessageHistory(
            table_name,
            session_id,
            session=session,
    )
    return _get_session_history

get_session_history = get_session_history_wrap(session, table_name)

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

A helper function, to make it easy to get answers.

In [ ]:
def ask(question, session_id=uuid.uuid4()):
    return conversational_rag_chain.invoke(
        {"input": question},
        config={
            "configurable": {"session_id": session_id,}
        },
    )["answer"]

# Sample usage
Let's see some examples, the usual ones.

## Ski boots

In [ ]:
id_ski_boots = str(uuid.uuid4())
ask("What is the name of the ski boots?", session_id=id_ski_boots)

In [ ]:
ask("Where have they been tested?", session_id=id_ski_boots)

In [ ]:
ask("What are they good for?", session_id=id_ski_boots)

## Downhill bike

In [ ]:
id_downhill_bike = str(uuid.uuid4())

ask("What is the name of the downhill bike?", session_id=id_downhill_bike)

In [ ]:
ask("What is it made of?", session_id=id_downhill_bike)

In [ ]:
ask("Who tested the bike?", session_id=id_downhill_bike)

In [ ]:
ask("What are they famous about?", session_id=id_downhill_bike)

## Infant bike


In [ ]:
id_infant_bike = str(uuid.uuid4())
ask("What is the max speed for the infant bike?", session_id=id_infant_bike)

In [ ]:
ask("Do I need any special tool for the assembly?", session_id=id_infant_bike)

In [ ]:
ask("How should I clean the bike?", session_id=id_infant_bike)

Done!